# Pdf to img

In [ ]:
!apt-get install -y poppler-utils

In [ ]:
# Install the required library
!pip install pdf2image

# Import necessary libraries
from pdf2image import convert_from_path
import os

# Function to convert PDF to images
def pdf_to_images(pdf_file, output_folder):
    # Check if the output folder exists, and create it if it doesn't
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert PDF to images
    images = convert_from_path(pdf_file)

    # Save each page as an image
    for i, image in enumerate(images):
        image_name = f"page_{i+1}.jpg"
        image_path = os.path.join(output_folder, image_name)
        image.save(image_path, "JPEG")
        print(f"Saved page {i+1} as {image_path}")

# Example usage
pdf_file = "/content/[ME] Ch-11 Kaiju No. 8 [@Manga_Edge].pdf"  # Replace with the path to your PDF file
output_folder = "/content/output_images"  # Replace with the desired output folder

pdf_to_images(pdf_file, output_folder)

# Artistic Coluring

In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
cd DeOldify

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

In [ ]:
!pip install -r requirements-colab.txt

In [ ]:
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True

In [ ]:

from deoldify.visualize import *
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [ ]:
#Adjust render_factor (int) if image doesn't look quite right (max 64 on 11GB GPU).  The default here works for most photos.
#It literally just is a number multiplied by 16 to get the square render resolution.
#Note that this doesn't affect the resolution of the final output- the output is the same resolution as the input.
#Example:  render_factor=21 => color is rendered at 16x21 = 336x336 px.
render_factor=35

In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

In [ ]:
colorizer = get_image_colorizer(artistic=True)

In [ ]:
# #NOTE:  Max is 45 with 11GB video cards. 35 is a good default
# render_factor=35

# source_path = '/content/output_images/page_10.jpg'
# result_path = None

# result_path = colorizer.plot_transformed_image(path=source_path, render_factor=render_factor, compare=False)

# show_image_in_notebook(result_path)

In [ ]:
import os
from pathlib import Path
from PIL import Image

# Set the render_factor (higher values produce better quality but take longer)
render_factor = 35

# Set the source and destination folders
source_folder = '/content/output_images/'
dest_folder = '/content/color_output/'

# Create the destination folder if it doesn't exist
os.makedirs(dest_folder, exist_ok=True)

# Loop through all files in the source folder
for filename in os.listdir(source_folder):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        source_path = os.path.join(source_folder, filename)

        # Colorize the image
        result_path = colorizer.plot_transformed_image(path=source_path, render_factor=render_factor, compare=True)

        # Save the colorized image to the destination folder
        dest_path = os.path.join(dest_folder, filename)
        image = Image.open(result_path)
        image.save(dest_path)
        print(f"Saved colorized image to {dest_path}")

# Color PDF

In [ ]:
!pip install img2pdf

In [ ]:
import os
from PIL import Image
from img2pdf import convert
import re

# Set the input and output paths
input_folder = '/content/color_output'
output_path = '/content/combined1.pdf'

# Get a list of all image files in the input folder
image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]

# Sort the image files based on the numerical order in their filenames
image_files.sort(key=lambda x: int(re.search(r'\d+', os.path.basename(x)).group()))

# Convert the list of image files to a PDF file
with open(output_path, 'wb') as pdf_file:
    pdf_file.write(convert(image_files))

print(f"Combined PDF saved to {output_path}")